---
layout: post
title:  Scala 2 Collections explained
date:   2025-10-27
categories: [Scala]
mermaid: true
maths: true
typora-root-url: /Users/ojitha/GitHub/ojitha.github.io
typora-copy-images-to: ../../blog/assets/images/${filename}
---

<style>
/* Styles for the two-column layout */
.image-text-container {
    display: flex; /* Enables flexbox */
    flex-wrap: wrap; /* Allows columns to stack on small screens */
    gap: 20px; /* Space between the image and text */
    align-items: center; /* Vertically centers content in columns */
    margin-bottom: 20px; /* Space below this section */
}

.image-column {
    flex: 1; /* Allows this column to grow */
    min-width: 250px; /* Minimum width for the image column before stacking */
    max-width: 40%; /* Maximum width for the image column to not take up too much space initially */
    box-sizing: border-box; /* Include padding/border in element's total width/height */
}

.text-column {
    flex: 2; /* Allows this column to grow more (e.g., twice as much as image-column) */
    min-width: 300px; /* Minimum width for the text column before stacking */
    box-sizing: border-box;
}

</style>

<div class="image-text-container">
    <div class="image-column">
        <img src="https://raw.githubusercontent.com/ojitha/blog/master/assets/images/2025-10-26-Scala2-Functors/scala-functors-illustration.svg" alt="Scala Functors" width="150" height="150">
    </div>
    <div class="text-column">
<p>TBC</p>
    </div>
</div>

<!--more-->

------

* TOC
{:toc}
------

## Introduction
Scala's collection library is one of its most powerful features, providing a rich set of data structures and operations for working with sequences, sets, and maps. The collections are designed to be **easy to use**, **concise**, **safe**, **fast**, and **universal**[^1].

## Collection Hierarchy

The Scala collections are organized into three main packages[^2]:

| Package | Description | Mutability |
|---------|-------------|------------|
| `scala.collection` | Base traits and abstract collections | May be immutable or mutable |
| `scala.collection.immutable` | Immutable collections (default) | Never change after creation |
| `scala.collection.mutable` | Mutable collections | Can be modified in place |

```mermaid
---
config:
  look: neo
  theme: default
---
graph TB
    Iterable[Iterable]
    Iterable --> Seq[Seq]
    Iterable --> Set[Set]
    Iterable --> Map[Map]
    
    Seq --> IndexedSeq[IndexedSeq]
    Seq --> LinearSeq[LinearSeq]
    
    IndexedSeq --> Vector
    IndexedSeq --> ArraySeq
    IndexedSeq --> Range
    IndexedSeq --> ArrayBuffer["ArrayBuffer (mutable)"]
    
    LinearSeq --> List
    LinearSeq --> LazyList
    LinearSeq --> Queue["Queue (immutable)"]
    
    Set --> SortedSet
    Set --> HashSet["HashSet"]
    Set --> BitSet
    SortedSet --> TreeSet
    
    Map --> SortedMap
    Map --> HashMap
    Map --> VectorMap
    SortedMap --> TreeMap
    
    style Iterable fill:#e1f5ff
    style Seq fill:#fff4e1
    style Set fill:#ffe1f5
    style Map fill:#e1ffe1
```

### The Iterable Trait

All Scala collections inherit from `Iterable[A]`, which defines an **iterator** that lets you loop through collection elements one at a time[^3]. The iterator can traverse the collection only once, as each element is consumed during iteration.

> **Important**: The `Iterable` trait provides the foundation for all collection operations through its iterator.

## Immutable vs. Mutable Collections

### Immutable Collections (Default)

Immutable collections **never change after creation**[^4]. When you "modify" an immutable collection, you create a new collection with the changes.

Immutable collections are the default:

In [3]:
val set = Set(1,2,3)
val list = List(1,2,3)
val map = Map(1 -> 'a', 2->'b')

set: Set[Int] = Set(1, 2, 3)
list: List[Int] = List(1, 2, 3)
map: Map[Int, Char] = Map(1 -> 'a')

Adding to immutable collections creates new collections:

In [5]:
val set2 = set + 4
val list2 = list :+ 4

set2: Set[Int] = Set(1, 2, 3, 4)
list2: List[Int] = List(1, 2, 3, 4)

Source📝[^5]: Demonstrates how immutable collections are the default in Scala and how they handle additions.

**Logic**: 
- First three lines create immutable collections without any import statements
- The `+` operator on Set and <b>`:+`</b>{:gtxt} operator on List create new collections rather than modifying the originals
- `set` and `list` remain unchanged after operations; only new bindings `set2` and `list2` contain the updated values

>Immutability ensures **referential transparency** - the same expression always evaluates to the same value, making code easier to reason about and thread-safe by default.
{:.green}

### Mutable Collections

Mutable collections can be modified in place using operations that have **side effects**[^6].

Must import or use full path for mutable collections:

In [7]:
import scala.collection.mutable

import scala.collection.mutable

In [13]:
val mutableSet = mutable.Set(1,2,3)

mutableSet: mutable.Set[Int] = HashSet(1, 2, 3)

In [9]:
val mutableList = mutable.ArrayBuffer(1,2,3)

mutableList: mutable.ArrayBuffer[Int] = ArrayBuffer(1, 2, 3)

In [11]:
val mutableMap = mutable.Map(1 -> 'a', 2 -> 'b')

mutableMap: mutable.Map[Int, Char] = HashMap(1 -> 'a', 2 -> 'b')

Modifying mutable collections in place:

In [15]:
mutableSet += 4
mutableList += 4

res14_0: mutable.Set[Int] = HashSet(1, 2, 3, 4)
res14_1: mutable.ArrayBuffer[Int] = ArrayBuffer(1, 2, 3, 4)

Source📝[^7]: Shows how mutable collections differ from immutable ones in both declaration and usage.

> Mutable collections use **side effects**{:rtxt} - they change state rather than creating new values. This can be more efficient for frequent updates but sacrifices thread-safety and referential transparency.
{:.yellow}

### Why Immutability Matters

**Benefits of Immutability**:
- **Thread-safe**: Multiple threads can safely access immutable collections
- **Easier to reason about**: No hidden state changes
- **Referential transparency**: Same inputs always produce same outputs
- **Structural sharing**: Efficient memory use through shared structure

## Main Collection Types

### 1. Sequences (Seq)

Sequences are ordered collections that support indexed access. They branch into two main categories:

#### Linear Sequences (LinearSeq)

Optimised for **head/tail** operations. Elements are accessed sequentially[^8].

```mermaid
---
config:
  look: handDrawn
  theme: default
---
graph LR
    List["List(1,2,3,4,5)"]
    N1[":: <br/> head: 1"]
    N2[":: <br/> head: 2"]
    N3[":: <br/> head: 3"]
    N4[":: <br/> head: 4"]
    N5[":: <br/> head: 5"]
    Nil["Nil"]
    
    List --> N1
    N1 -->|tail| N2
    N2 -->|tail| N3
    N3 -->|tail| N4
    N4 -->|tail| N5
    N5 -->|tail| Nil
```

The `List` is implemented as a *singly linked list* where each node contains a value and a pointer to the next node. This structure makes prepending (`::`), head, and tail operations extremely fast, but random access requires traversing from the beginning.

**List characteristics**:
- **cons (::)** operation prepends elements in O(1) time
- Random access is O(n) - must traverse from head
- Ideal for recursive algorithms and pattern matching

In [16]:
val list = List(1, 2, 3, 4, 5)

list: List[Int] = List(1, 2, 3, 4, 5)

Head and tail operations are O(1)

In [17]:
list.head
list.tail

res16_0: Int = 1
res16_1: List[Int] = List(2, 3, 4, 5)

In [18]:
list.isEmpty

res17: Boolean = false

Pattern matching with Lists:

In [20]:
list match {
    case h :: t => println(s"Head: $h")
    case Nil => println("Empty list")
}

Head: 1


Example📝[^9]: Demonstrates the fundamental operations on List - a singly linked list optimised for sequential access.

- `head` extracts the first element in constant O(1) time
- `tail` returns all elements except the first, also in O(1) time
- `::` (cons) operator in pattern matching destructures the list into head and tail
- `Nil` represents the empty list

#### Indexed Sequences (IndexedSeq)

Optimised for **random access**. Elements can be accessed efficiently by index[^10].
Vector - the default indexed sequence

In [21]:
val vector = Vector(1,2,3,4,5)

vector: Vector[Int] = Vector(1, 2, 3, 4, 5)

Optimized for **random access**. Elements can be accessed efficiently by index[^10].

In [24]:
val indexed = IndexedSeq(1,2,3)

indexed: IndexedSeq[Int] = Vector(1, 2, 3)

In [25]:
val updated = vector.updated(2, 22)

updated: Vector[Int] = Vector(1, 2, 22, 4, 5)

Example📝[^11]: Shows Vector's strength in random access and updates compared to List.

**Data Structure**: Vector uses a **tree structure**. This gives effectively O(1) access time for practical purposes. Updates use **structural sharing** - only the path from root to the changed element is copied.

-   **Branching factor of 32** - Each internal node can have up to 32 children
-   **Shallow and wide** - Typical depth is 0-6 levels even for millions of elements
-   **Array-based nodes** - Each node is backed by an array of size 32
-   **Efficient access** - $O(\log_{32} n)$ ≈ effectively constant time for practical sizes
-   **Persistent/Immutable** - Structural sharing enables efficient copying

```mermaid
---
config:
  look: handDrawn
  theme: default
---
graph TD
    V["Vector(1,2,3,4,5,6,7,8)"]
    Root["Root Node<br/>(Internal)"]
    
    L1["Leaf Array<br/>[1, 2, 3, 4]"]
    L2["Leaf Array<br/>[5, 6, 7, 8]"]
    
    V --> Root
    Root --> L1
    Root --> L2
    
    style Root fill:#e1f5ff
    style L1 fill:#c8e6c9
    style L2 fill:#c8e6c9
```

### 2. Sets

Sets are collections of **unique elements** with no defined order (except for sorted sets)[^12].
```mermaid
---
config:
  look: neo
  theme: default
---
classDiagram
    Set <|-- HashSet
    Set <|-- SortedSet
    Set <|-- BitSet
    SortedSet <|-- TreeSet
    
    class Set {
        +contains(elem): Boolean
        ++(elem): Set
        --(elem): Set
        union(other): Set
        intersect(other): Set
        diff(other): Set
    }
    
    class HashSet {
        Hash-based storage
        O(1) lookup
    }
    
    class TreeSet {
        Red-black tree
        Sorted order
        O(log n) operations
    }
    
    class BitSet {
        Bit array
        Non-negative integers only
        Memory efficient
    }
```
#### Immutable Sets
Creating immutable sets

In [26]:
val set1 = Set(1,2,3)
val set2 = Set(3,4,5)

set1: Set[Int] = Set(1, 2, 3)
set2: Set[Int] = Set(3, 4, 5)

Adding elements (returns a new set):

In [28]:
val set3 = set1 + 4
val set4 = set1 ++ List(5,6)

set3: Set[Int] = Set(1, 2, 3, 4)
set4: Set[Int] = HashSet(5, 1, 6, 2, 3)

In [32]:
println(s"set4 is a: ${set4.getClass.getSimpleName}")

set4 is a: HashSet


Scala's immutable HashSet uses a Hash Array Mapped Trie (HAMT) - a tree structure optimised for hash-based lookups.

```mermaid
---
config:
  look: handDrawn
  theme: default
---
graph TD
    Root["Root Node<br/>(bitmap: 32-bit)"]
    
    subgraph "Level 1 - First 5 bits of hash"
    N0["Node at index 0"]
    N5["Node at index 5"]
    N17["Node at index 17"]
    end
    
    subgraph "Level 2 - Next 5 bits of hash"
    N5_3["Node at index 3"]
    N5_12["Node at index 12"]
    end
    
    subgraph "Leaf Level"
    L1["Value: 1<br/>hash: 0b00000..."]
    L2["Value: 2<br/>hash: 0b00101..."]
    L3["Value: 5<br/>hash: 0b00101..."]
    L4["Value: 17<br/>hash: 0b10001..."]
    end
    
    Root -->|bits 0-4| N0
    Root -->|bits 0-4| N5
    Root -->|bits 0-4| N17
    
    N5 --> N5_3
    N5 --> N5_12
    
    N0 --> L1
    N5_3 --> L2
    N5_12 --> L3
    N17 --> L4
    
    style Root fill:#e1f5ff
    style N0 fill:#fff9c4
    style N5 fill:#fff9c4
    style N17 fill:#fff9c4
    style N5_3 fill:#fff9c4
    style N5_12 fill:#fff9c4
    style L1 fill:#c8e6c9
    style L2 fill:#c8e6c9
    style L3 fill:#c8e6c9
    style L4 fill:#c8e6c9
```

Key Characteristics:

1. **Hash-based Indexing**
    - Each element's hash code determines its position
    - Hash code is split into 5-bit chunks (32 = 2⁵ branches per level)
    - Each chunk selects which child node to follow
2. **Bitmap Compression**
    - Each node has a 32-bit bitmap indicating which children exist
    - Only allocated children are stored (sparse representation)
    - Saves memory for sparse hash distributions
3. **Tree Properties**
    - Branching factor: 32 (2⁵)
    - Max depth: ~6-7 levels for millions of elements
    - Time complexity: $$O(\log_{32} n)$$ ≈ O(1) effectively
    - Space: Efficient with structural sharing

Performance Characteristics:

| Operation | Time Complexity | Explanation |
|-----------|----------------|-------------|
| `contains` | $O(\log_{32} n) \approx O(1)$ | Hash lookup with shallow tree |
| `+` (add) | $O(\log_{32} n) \approx O(1)$ | Copy path, share rest |
| `++` (union) | $O(m \log_{32} n)$ | Add $m$ elements |
| `size` | $O(1)$ | Cached |

Set operations:

In [33]:
set1 union set2

res32: Set[Int] = HashSet(5, 1, 2, 3, 4)

In [34]:
set1 intersect set2

res33: Set[Int] = Set(3)

In [35]:
set1 diff set2

res34: Set[Int] = Set(1, 2)

Membership testing $O(1)$ for HashSet

In [36]:
set1.contains(1)
// alternatively
set1(1)

res35_0: Boolean = true
res35_1: Boolean = true

Source📝[^13]: Demonstrates basic Set operations and their mathematical set theory foundations.

**Logic**:
- `+` and `++` add elements, creating new sets (immutable)
- `union` combines all elements from both sets (mathematical ∪ operator)
- `intersect` keeps only common elements (mathematical ∩ operator)
- `diff` returns elements in first set but not in second (mathematical \ operator)
- `contains` checks membership, implemented as hash lookup for O(1) performance

Sets implement mathematical set operations:

**Union**: 
$$A ∪ B = \{x | x ∈ A \text{ or } x ∈ B\}$$

**Intersection**: 
$$A ∩ B = \{x | x ∈ A \text{ and } x ∈ B\}$$

**Difference**: 
$$A \setminus B = \{x | x ∈ A \text{ and } x ∉ B\}$$

The uniqueness property ensures no duplicates: adding existing elements has no effect.

#### Sorted Sets
TreeSet maintains elements in sorted order

In [37]:
val sortedSet = scala.collection.immutable.TreeSet(5, 2, 8, 1, 9)

sortedSet: collection.immutable.TreeSet[Int] = TreeSet(1, 2, 5, 8, 9)

The `TreeSet` operation creates an immutable, sorted set containing the given elements. The result stores unique values in ascending order according to an implicit `Ordering`.

```mermaid
---
config:
  look: neo
  theme: default
---
graph TD
    A["TreeSet(5, 2, 8, 1, 9)"] -->|"Underlying Structure"| B["Red-Black Tree"]
    B -->|"Balanced BST"| C["Operations: O(log n)"]
    C -->|"Insert"| D["O(log n)"]
    C -->|"Delete"| E["O(log n)"]
    C -->|"Search"| F["O(log n)"]
    
    G["Tree Structure:<br/>5 is root"] -->|"left subtree"| H["2"]
    G -->|"right subtree"| I["8"]
    H -->|"left"| J["1"]
    I -->|"right"| K["9"]
    
    style B fill:#e1f5ff
    style C fill:#fff3e0
    style D fill:#f3e5f5
    style E fill:#f3e5f5
    style F fill:#f3e5f5
```

Red-Black Tree Properties

A Red-Black Tree is a self-balancing binary search tree with these invariants:

1. **Every node is colored red or black** - Ensures the tree remains relatively balanced
2. **The root is always black** - Simplifies the tree structure
3. **Red nodes have only black children** - Prevents consecutive red nodes
4. **Every path from root to null has the same number of black nodes** - Guarantees O(log n) height

These properties ensure that the tree height is at most 2 × log(n + 1), guaranteeing logarithmic complexity for all operations.

Algorithmic Complexity Analysis

| Operation | Time Complexity | Space Complexity | Description |
|-----------|-----------------|------------------|-------------|
| **Construction** | O(n log n) | O(n) | Creating TreeSet from n elements requires n insertions, each O(log n) |
| **Insertion** | O(log n) | O(1) amortized | Adding an element requires tree rotations and rebalancing |
| **Deletion** | O(log n) | O(1) amortized | Removing an element requires tree rotations and rebalancing |
| **Search/Contains** | O(log n) | O(1) | Binary search in balanced tree structure |
| **Iteration** | O(n) | O(1) | In-order traversal visits each node once |
| **Minimum/Maximum** | O(log n) | O(1) | Must traverse to leftmost/rightmost leaf |

First and last operations

In [38]:
sortedSet.head
sortedSet.last

res37_0: Int = 1
res37_1: Int = 9

Range queries

In [39]:
sortedSet.range(2, 8)

res38: collection.immutable.TreeSet[Int] = TreeSet(2, 5)

Example📝[^14]

Why Red-Black Tree for TreeSet?

When to Use TreeSet

| Use TreeSet When | Don't Use TreeSet When |
|-----------------|----------------------|
| You need sorted iteration | Elements don't need ordering |
| You need range queries (headSet, tailSet) | You frequently add/remove elements |
| Thread safety is important | Mutability would improve performance |
| You want guaranteed O(log n) operations | You need O(1) average case (use HashSet) |
| Working with small datasets | Performance overhead is critical |

```mermaid
---
config:
  look: neo
  theme: default
---
graph LR
    A["Scala Sets"] -->|"Immutable"| B["HashSet"]
    A -->|"Immutable"| C["TreeSet"]
    A -->|"Immutable"| D["BitSet"]
    A -->|"Mutable"| E["scala.collection.mutable<br/>HashSet"]
    A -->|"Mutable"| F["scala.collection.mutable<br/>TreeSet"]
    
    B -->|"Complexity"| B1["avg O(1)<br/>worst O(n)"]
    C -->|"Complexity"| C1["O(log n)<br/>guaranteed"]
    
    B -->|"Order"| B2["Unordered"]
    C -->|"Order"| C2["Sorted"]
    
    style A fill:#e0f2f1
    style C fill:#fff9c4
    style B fill:#f3e5f5
```

### 3. Maps

Maps store **key-value pairs** where all keys must be unique[^15].

```mermaid
---
config:
  look: neo
  theme: default
---
classDiagram
    Map <|-- HashMap
    Map <|-- SortedMap
    Map <|-- VectorMap
    SortedMap <|-- TreeMap
    
    class Map {
        +get(key): Option[Value]
        +apply(key): Value
        +getOrElse(key, default): Value
        +(key -> value): Map
        -(key): Map
    }
    
    class HashMap {
        Hash trie storage
        O(1) lookup
        No ordering
    }
    
    class TreeMap {
        Red-black tree
        Sorted by keys
        O(log n) operations
    }
    
    class VectorMap {
        Preserves insertion order
        Vector-based
    }
```

#### Creating and Using Maps

In [45]:
val map1 = Map(1 -> "a", 2 -> "b", 3 -> "c")
println(s"map1 is a: ${map1.getClass.getSimpleName}")

map1 is a: Map3


map1: Map[Int, String] = Map(1 -> "a", 2 -> "b", 3 -> "c")

Alternative syntax

In [44]:
val map2 = Map((1,"a"), (2, "b"), (3, "c"))
println(s"map2 is a: ${map2.getClass.getSimpleName}")

map2 is a: Map3


map2: Map[Int, String] = Map(1 -> "a", 2 -> "b", 3 -> "c")

Accessing values

In [47]:
map1(1)

res46: String = "a"

if key not found

In [48]:
map1(4)

: 

**Option Pattern**: The `get` method returns `Option[V]` instead of throwing exceptions, following functional programming's principle of making errors explicit in types:
- `Some(value)` when key exists
- `None` when key doesn't exist

In [50]:
map1.get(1)
map1.get(4)
map1.getOrElse(4, '?')

res49_0: Option[String] = Some(value = "a")
res49_1: Option[String] = None
res49_2: Any = '?'

Adding/updating entries (immutable)

In [52]:
map1 + (4 -> "d")

res51: Map[Int, String] = Map(1 -> "a", 2 -> "b", 3 -> "c", 4 -> "d")

Removing entries by the key:

In [53]:
map1 - 2 // 2 is a key

res52: Map[Int, String] = Map(1 -> "a", 3 -> "c")

Source📝[^16] Shows different ways to create, access, and modify immutable Maps.
- `->` operator creates tuple pairs (syntactic sugar for `(1, "a")`)
- `apply(key)` throws exception if key doesn't exist (unsafe but concise)
- `get(key)` returns `Option[Value]` - safe handling of missing keys
- `getOrElse(key, default)` provides fallback value for missing keys
- `+` and `++` create new maps with additional entries
- `-` creates new map with key removed

This forces callers to handle the missing key case, preventing runtime errors.

#### Traversing Maps
**For comprehension** with tuple destructuring: `(k, v) <- ratings` automatically unpacks each key-value pair

In [57]:
val map1 = Map(1 -> "a", 2 -> "b", 3 -> "c")

for ((k, v) <- map1) { println(s"key: $k and value is: $v") }

key: 1 and value is: a
key: 2 and value is: b
key: 3 and value is: c


map1: Map[Int, String] = Map(1 -> "a", 2 -> "b", 3 -> "c")

**Pattern matching** in foreach: `case (movie, rating) =>` explicitly shows tuple destructuring

In [60]:
map1.foreach{
    case (k, v) => println(s"key: $k and value is: $v")
}

key: 1 and value is: a
key: 2 and value is: b
key: 3 and value is: c


**Tuple accessors**: `x._1` (key) and `x._2` (value) directly access tuple components

In [61]:
map1.foreach(x => println(s"key: ${x._1} and value is ${x._2}"))

key: 1 and value is a
key: 2 and value is b
key: 3 and value is c


**Separate iteration**: `keys` and `values` methods provide iterators over each component independently

In [66]:
map1.keys.foreach(k => print(k))
map1.values.foreach(v => print(v))

123abc

Source📝[^17]: Demonstrates multiple idiomatic ways to traverse Map entries.

> Maps are Iterables of key-value pairs (also named mappings or associations)
{:.info-box}

**Map as Collection of Tuples**: Maps are conceptually `Iterable[(K, V)]` - a collection of 2-tuples. Each iteration yields a pair `(key, value)` which can be destructured using pattern matching or accessed via tuple syntax.

**Performance Note**: Using `keys` or `values` creates lightweight iterators without copying data.

**Transformer methods** take at least one collection and produce a new collection[^18]. In Scala 2, these include:

| Method | Description | Example |
|--------|-------------|---------|
| `map` | Apply function to each element | `List(1,2,3).map(_ * 2)` → `List(2,4,6)` |
| `filter` | Keep elements matching predicate | `List(1,2,3,4).filter(_ > 2)` → `List(3,4)` |
| `flatMap` | Map and flatten nested results | `List(1,2).flatMap(n => List(n, n*10))` → `List(1,10,2,20)` |
| `collect` | Partial function mapping + filtering | `List(1,2,3).collect { case x if x > 1 => x * 2 }` → `List(4,6)` |

#### The map Method
The `map(_ * 2)` uses underscore syntax for anonymous function: `x => x * 2`

In [67]:
val numbers = List(1,2,3,4,5)
numbers.map(_ * 2)

numbers: List[Int] = List(1, 2, 3, 4, 5)
res66_1: List[Int] = List(2, 4, 6, 8, 10)

map with function

In [68]:
List("a", "bb", "ccc").map(_.length)

res67: List[Int] = List(1, 2, 3)

map preserves collection type

In [69]:
val vector = Vector(1, 2, 3)
val vectorResult = vector.map(_ + 1)

vector: Vector[Int] = Vector(1, 2, 3)
vectorResult: Vector[Int] = Vector(2, 3, 4)

Example📝[^19]

**Functor Laws**: The `map` operation makes collections into **functors**[^52]. To be a valid functor, `map` must satisfy two laws:

1. **Identity**: `collection.map(x => x) == collection`
   - Mapping the identity function returns the same collection

2. **Composition**: `collection.map(f).map(g) == collection.map(x => g(f(x)))`
   - Mapping twice equals mapping the composition once
   - This means: `map(f).map(g)` = `map(g ∘ f)` where `∘` is function composition

**Mathematics**: If we have functions $f: A → B$ and $g: B → C$, then:
$$\text{map}(g) ∘ \text{map}(f) = \text{map}(g ∘ f)$$

This property allows the compiler to optimize chained `map` operations.

#### The filter Method
The `filter(_ % 2 == 0)` keeps only elements where the predicate returns `true`: $$\{x \in \text{numbers} | x \bmod 2 = 0\}$$

In [73]:
val numbers = (1 to 10).toList
// Filter even numbers
numbers.filter( _ % 2 == 0 )

numbers: List[Int] = List(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
res72_1: List[Int] = List(2, 4, 6, 8, 10)

Multiple `filter` calls can be chained: `filter(p1).filter(p2)` ≡ `filter(x => p1(x) && p2(x))`

In [74]:
numbers.filter(_ > 2).filter(_ < 8)

res73: List[Int] = List(3, 4, 5, 6, 7)

Example📝[^21]: Demonstrates filtering collections based on boolean predicates (test conditions).

**Predicate Function**: A **predicate** is a function that returns a Boolean: `A => Boolean`. It answers "yes" or "no" questions about each element.

Filter implements set comprehension notation:
$$\{x \in S | P(x)\}$$

Read as: "the set of all x in S such that P(x) is true"


**Performance**: Filter must traverse all elements once, giving O(n) time complexity. The resulting collection may be smaller but never larger than the input.

#### The flatMap Method

Imagine you have a **wrapped value** inside a container (like `Option[Int]` or `List[String]`). You want to:

1. **Extract** the value from its container
2. **Transform** it using a function that *also* produces a wrapped result
3. **Avoid nesting** containers (so you don't end up with `Option[Option[Int]]`)

This is exactly what `flatMap` does. The word "flat" is key—it flattens the nesting that would naturally occur.

```
flatMap: M[A] → (A → M[B]) → M[B]
```

| Symbol | Meaning |
|--------|---------|
| M[A] | A value of type A wrapped in monad M (e.g., Option[Int], List[String]) |
| A | The unwrapped value type |
| A → M[B] | A function that takes a plain A and returns a wrapped B |
| M[B] | The final result: a B wrapped in monad M (same monad, different type) |

In Scala syntax:

```scala
def flatMap[A, B](ma: M[A])(f: A => M[B]): M[B]
```

For `flatMap` to preserve monadic semantics, it must satisfy  3 **Monad Laws**:

**Left Identity**: Creating a context from a value and then flatMap-ing is the same as just applying the function:

```
unit(a).flatMap(f) = f(a)
```

**Right Identity**: flatMap-ing with the constructor does nothing:

```
m.flatMap(unit) = m
```

**Associativity**: Chaining two sequential flatMap calls is the same as nesting one inside the other:

```
m.flatMap(f).flatMap(g) = m.flatMap(x => f(x).flatMap(g))
```

For example, Monad Type Class Definition:
- `pure[A](value: A): F[A]` — wraps a plain value in monad F
- `flatMap[A, B](ma: F[A])(f: A => F[B]): F[B]` — sequences operations
- `map` is derived from `flatMap` + `pure` — all monads automatically get `map`

In [3]:
import scala.language.higherKinds

trait Monad[F[_]] {
  def pure[A](value: A): F[A]
  def flatMap[A, B](ma: F[A])(f: A => F[B]): F[B]
  def map[A, B](ma: F[A])(f: A => B): F[B] =
    flatMap(ma)(a => pure(f(a)))
}

implicit val optionMonad: Monad[Option] = new Monad[Option] {
  def pure[A](value: A): Option[A] = Some(value)
  def flatMap[A, B](ma: Option[A])(f: A => Option[B]): Option[B] = ma.flatMap(f)
}

implicit val listMonad: Monad[List] = new Monad[List] {
  def pure[A](value: A): List[A] = List(value)
  def flatMap[A, B](ma: List[A])(f: A => List[B]): List[B] = ma.flatMap(f)
}

def combine[M[_]: Monad](a: M[Int], b: M[Int])(implicit M: Monad[M]): M[Int] =
  M.flatMap(a) { x =>
    M.flatMap(b) { y =>
      M.pure(x + y)
    }
  }

// Explicitly specify type parameter
combine[Option](Some(3), Some(4))       // Some(7) ✅
combine[List](List(1, 2), List(3, 4))   // List(4, 5, 5, 6) ✅

import scala.language.higherKinds


defined trait Monad
optionMonad: Monad[Option] = ammonite.$sess.cmd2$Helper$$anon$1@7c758c02
listMonad: Monad[List] = ammonite.$sess.cmd2$Helper$$anon$2@650110e5
defined function combine
res2_5: Option[Int] = Some(value = 7)
res2_6: List[Int] = List(4, 5, 5, 6)

Source: *Advanced Scala*, Chapter 4: "Monads" → Section 4.1: "Monad Definition and Laws"

For lists, flatMap implements **list comprehensions**:
$$[f(x, y) | x ← xs, y ← ys] = \text{xs.flatMap}(x ⇒ \text{ys.map}(y ⇒ f(x, y)))$$

In [75]:
val numbers = (1 to 3).toList

numbers: List[Int] = List(1, 2, 3)

In [77]:
numbers.map(n => 2 * n)

res76: List[Int] = List(2, 4, 6)

The `map` with a function returning collections creates nested structure: `List[List[Int]]`

In [78]:
val nested = numbers.map(n => List(n, n * 10))

nested: List[List[Int]] = List(List(1, 10), List(2, 20), List(3, 30))

The `flatMap` applies the function **and** flattens one level: `List[Int]`

In [84]:
nested.flatten // flatten the above nested 
numbers.flatMap(n => List(n, n * 10)) // ✔️

res83_0: List[Int] = List(1, 10, 2, 20, 3, 30)
res83_1: List[Int] = List(1, 10, 2, 20, 3, 30)

Example📝[^22]: Shows how `flatMap` combines mapping and flattening into one operation, crucial for working with nested structures and monadic types.

```mermaid
---
config:
  look: neo
  theme: default
---
graph TD
    A["numbers = List(1, 2, 3)"] -->|"map(n => List(n, n*10))"| B["List(List(1,10), List(2,20), List(3,30))"]
    
    B -->|"flatten"| C["List(1, 10, 2, 20, 3, 30)"]
    
    A -->|"flatMap(n => List(n, n*10))"| D["List(1, 10, 2, 20, 3, 30)"]
    
    C -->|"same result"| D
    
    style A fill:#e3f2fd
    style B fill:#fff3e0
    style C fill:#c8e6c9
    style D fill:#c8e6c9
```

- `map` with a function returning collections creates nested structure: `List[List[Int]]`
- `flatMap` applies the function **and** flattens one level: `List[Int]`


**Relationship**: `flatMap(f)` ≡ `map(f).flatten`

**Monadic Bind**: `flatMap` is the **bind** operation in monadic programming. For a monad `M[A]`:
$$\text{flatMap}: M[A] → (A → M[B]) → M[B]$$

This allows **sequencing computations** where each step may produce multiple results (List) or may fail (Option).

With `Option`, `flatMap` filters out `None` values automatically:
  - `Some(x)` → contributes `x` to result
  - `None` → contributes nothing (filtered out)

In [5]:
def findInt(s: String): Option[Int] = 
    try Some(s.toInt)
    catch {case _: NumberFormatException => None
}

defined function findInt

test the above method:

In [8]:
val strings = List("2", "too", "5.22", "two", "10")
strings.flatMap(findInt)

strings: List[String] = List("2", "too", "5.22", "two", "10")
res7_1: List[Int] = List(2, 10)

A monad is a control mechanism for sequencing computations. Instead of thinking of it as a purely mathematical concept, think of it as a way to specify "what happens next" in a computation, where the next step may depend on the result of the previous step.

```scala
def flatMap[A, B](ma: M[A])(f: A => M[B]): M[B]
```

### Reduction Operations

Operations that **combine** elements to produce a single result:

#### fold, foldLeft, foldRight

Fold operations implement **catamorphisms**, which are generalisations of recursion. They "tear down" a structure to a single value.

For a list `[a₁, a₂, a₃, a₄]` with operation `⊕` and initial value `z`:

**foldLeft** (left-associative):
$$((((z ⊕ a₁) ⊕ a₂) ⊕ a₃) ⊕ a₄)$$

**foldRight** (right-associative):
$$(a₁ ⊕ (a₂ ⊕ (a₃ ⊕ (a₄ ⊕ z))))$$

**Associativity Requirement for fold**: Operation must satisfy:
$$(a ⊕ b) ⊕ c = a ⊕ (b ⊕ c)$$

Examples: `+`, `*`, `max`, `min` are associative; `-`, `/` are not.


In [9]:
val numbers = (1 to 5).toList

numbers: List[Int] = List(1, 2, 3, 4, 5)

**foldLeft**: Processes left-to-right with accumulator as first parameter: `((((z ⊕ a₁) ⊕ a₂) ⊕ a₃) ⊕ a₄)`

> ```scala
> def foldLeft[B](z: B)(op: (B, A) => B): B   // z is accumulator, A is element
> ```
{:.info-box}

In [16]:
numbers.foldLeft(0)(_ + _)
// 0 + 1 = 1, 1 + 2 = 3, 3 + 3 = 6, 6 + 4 = 10, 10 + 5 = 15

//with an accumulator
numbers.foldLeft("")((acc, n) => acc + n)

res15_0: Int = 15
res15_1: String = "12345"

**foldRight**: Processes right-to-left with accumulator as second parameter: `(a₁ ⊕ (a₂ ⊕ (a₃ ⊕ (a₄ ⊕ z))))`

> ```scala
> def foldRight[B](z: B)(op: (A, B) => B): B  // z is accumulator, A is element
> ```
{:.info-box}

In [14]:
numbers.foldRight(List.empty[Int])((elem, acc) => elem :: acc)

res13: List[Int] = List(1, 2, 3, 4, 5)

**fold**: Can process in any order (requires associativity), enables parallelization
> ```scala
> def fold[A1 >: A](z: A1)(op: (A1, A1) => A1): A1
> ```
{:.info-box}

In [15]:
numbers.fold(1)(_ * _)

res14: Int = 120

Source📝[^23]: Demonstrates folding operations that reduce a collection to a single value by repeatedly applying a binary operation.

**Subtraction Example Analysis**:
- `reduceLeft(_ - _)` on `[1,2,3,4,5]`: 
  - `((((1 - 2) - 3) - 4) - 5)` = `(((-1 - 3) - 4) - 5)` = `((-4 - 4) - 5)` = `(-8 - 5)` = `-13`
  
- `reduceRight(_ - _)` on `[1,2,3,4,5]`:
  - `(1 - (2 - (3 - (4 - 5))))` = `(1 - (2 - (3 - (-1))))` = `(1 - (2 - 4))` = `(1 - (-2))` = `3`
 
**Key Differences from fold**:
| Feature | fold | reduce |
|---------|------|--------|
| Initial value | Required parameter | Uses first element |
| Empty collection | Returns initial value | Throws exception (use reduceOption) |
| Result type | Can differ from element type | Must be same as element type |

**When to use reduce**: When the operation is associative and you don't need a different result type. The first element naturally serves as the identity element.

### Collection Queries

#### Checking Conditions


In [17]:
val numbers = (1 to 5).toList
numbers.exists(_ > 4)

numbers: List[Int] = List(1, 2, 3, 4, 5)
res16_1: Boolean = true

## References

[^1]: *Programming in Scala, Fourth Edition*, Ch. 24: "Collections in Depth" → "Introduction", p. 573

[^2]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Understanding the Collections Hierarchy", p. 318-319

[^3]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Understanding the Collections Hierarchy", p. 320

[^4]: *Programming in Scala, Fourth Edition*, Ch. 24: "Collections in Depth" → "Collections Hierarchy", p. 574

[^5]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Understanding the Collections Hierarchy", p. 323

[^6]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Understanding the Collections Hierarchy", p. 322

[^7]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Understanding the Collections Hierarchy", p. 322

[^8]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Sequences", p. 321

[^9]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "List Operations"

[^10]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Sequences", p. 321-322

[^11]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Understanding the Collections Hierarchy", p. 320

[^12]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Sets", p. 323

[^13]: *Scala Cookbook, Second Edition*, Ch. 15: "Collections: Tuple, Range, Set, Stack, and Queue" → "Creating a Set", p. 453-454

[^14]: *Scala Cookbook, Second Edition*, Ch. 15: "Collections: Tuple, Range, Set, Stack, and Queue" → "Sorted Sets"

[^15]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Maps", p. 322

[^16]: *Scala Cookbook, Second Edition*, Ch. 14: "Collections: Using Maps" → "Creating and Using Maps", p. 424

[^17]: *Scala Cookbook, Second Edition*, Ch. 14: "Collections: Using Maps" → "Traversing a Map", p. 435-436

[^18]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Transformer Methods", p. 371

[^19]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "map Method", p. 387-389

[^20]: *Programming in Scala, Fourth Edition*, Ch. 24: "Collections in Depth" → "Template Traits", p. 588

[^21]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "filter Method", p. 392-394

[^22]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "flatMap Method"

[^23]: *Functional Programming in Scala*, Ch. 10: "Monoids" → "Folding", p. 180

[^24]: *Functional Programming in Scala*, Ch. 10: "Monoids" → "Associativity and parallelism", p. 179-181

[^25]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "reduce Method", p. 400-405

[^26]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Collection Methods"

[^27]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Taking and Dropping"

[^28]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Grouping Methods", p. 397-399

[^29]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Sorting", p. 410-415

[^30]: *Programming in Scala, Fourth Edition*, Ch. 23: "For Expressions Revisited" → "Translation", p. 557-561

[^31]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "for/yield with Collections", p. 387-390

[^32]: *Programming in Scala, Fourth Edition*, Ch. 23: "For Expressions Revisited" → "Monads", p. 566-567

[^33]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Views", p. 335-337

[^34]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Views", p. 336

[^35]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Performance", p. 330-333

[^36]: *Scala Cookbook, Second Edition*, Ch. 23: "Types" → "Variance", p. 670-674

[^37]: *Scala Cookbook, Second Edition*, Ch. 23: "Types" → "Covariance", p. 671-672

[^38]: *Programming in Scala, Fourth Edition*, Ch. 19: "Type Parameterization" → "Contravariance", p. 425-426

[^39]: *Programming in Scala, Fourth Edition*, Ch. 19: "Type Parameterization" → "Function Variance", p. 426-427

[^40]: *Programming in Scala, Fourth Edition*, Ch. 19: "Type Parameterization" → "Liskov Substitution Principle", p. 425

[^41]: *Programming in Scala, Fourth Edition*, Ch. 19: "Type Parameterization" → "Arrays and Variance", p. 422-423

[^42]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Iterators", p. 381-384

[^43]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Iterators", p. 381-382

[^44]: *Scala Cookbook, Second Edition*, Ch. 15: "Collections: Tuple, Range, Set, Stack, and Queue" → "Ranges", p. 449-453

[^45]: *Scala Cookbook, Second Edition*, Ch. 15: "Collections: Tuple, Range, Set, Stack, and Queue" → "Ranges", p. 452-453

[^46]: *Advanced Scala with Cats*, Ch. 3: "Functors" → "Maybe Monad"

[^47]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Option Integration"

[^48]: Philip Wadler, "Comprehending Monads", p. 22, *WadlerMonads.pdf*

[^49]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → Examples

[^50]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "flatMap examples"

[^51]: *Scala Cookbook, Second Edition*, Ch. 14: "Collections: Using Maps" → "Immutable Map Updates"

[^52]: [Scala 2 Functors Explained]({% link _posts/2025-10-26-Scala2-Functors.md %}){:target="_blank"}

{:gtxt: .message color="green"}

{:ytxt: .message color="yellow"}

{:rtxt: .message color="red"}